## D'NOVA YouTube Shorts Scraping Flow
-  Test code: 테스트 코드
- Daily Shorts check: 수집 데이터 loading
#### Code Flow
1. 기존 수집했던 쇼츠 재수집 -7d, -14d, -21d
2. channel shorts url input으로 들어갈시 채널 url 필터링(daily)
3. 수집한 channel list 수집 후 loaded_db에 적재 

### Test Code

In [5]:
import requests
import re
import numpy as np
from bs4 import BeautifulSoup

In [145]:
# Define the payload and headers for the request
# shorts channel1: https://www.youtube.com/watch?v=xEuH_5Ik92U
# shorts channel2: https://www.youtube.com/watch?v=fa_sSaHxTZ4
# yt_shorts page: https://www.youtube.com/@%EC%95%84%EC%9D%B4%EB%8F%8C/shorts
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
payload = {
    'api_key': API_KEY,
    'url': 'https://www.youtube.com/watch?v=fa_sSaHxTZ4'
    ,'render': 'true'
}
# payload = { 'api_key': '3397e8a6a37e89b03b08750a27575df8', 'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U' }
# r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
# print(r.text)
headers = {
    'x-sapi-render': 'true',
    'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 3}]'
    # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
    # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
}
# Make the GET request
r_full = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False) # headers=headers,

# Save the response text to a file
with open('response_channel_test.txt', 'w', encoding='utf-8') as file:
    file.write(r_full.text)

print("Response saved to response_output.txt")


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 

In [161]:
# url 한개만 추출 -> comments (x)
import requests

payload = { 'api_key': '3397e8a6a37e89b03b08750a27575df8', 'url': 'https://www.youtube.com/watch?v=PUs_h8-PDeg'}
r_normal = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
# Save the response text to a file
with open('response_channel_test_예와_Rovert.txt', 'w', encoding='utf-8') as file:
    file.write(r_normal.text)

print("Response saved to response_output.txt")

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response saved to response_output.txt


In [112]:
# url 여러개 진행
import requests
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# List of URLs to test
urls = [
    'https://www.youtube.com/watch?v=xEuH_5Ik92U',
    'https://www.youtube.com/watch?v=CPsCEfs3m8I',
    'https://www.youtube.com/watch?v=jXUHFbNo5Vg'
]

# API key and headers
headers = {
    'x-sapi-render': 'true',
    # 'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
    'x-sapi-instruction_set': '[{"type":"loop","for":' + str(len(urls))+',"instructions":[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]}]'
}

# Loop through each URL, send request, and save response to a file
for i, url in enumerate(urls):
    payload = {
        'api_key': API_KEY,
        'url': url,
        'render': 'true'
    }
    
    # Send the request
    r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
    
    # Save the response to a file, naming it based on the URL index
    filename = f'response_{i + 1}.txt'
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(r.text)
    
    print(f'Response for {url} saved to {filename}')



c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=xEuH_5Ik92U saved to response_1.txt


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=CPsCEfs3m8I saved to response_2.txt


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=jXUHFbNo5Vg saved to response_3.txt


### Daily Shorts Check
- 95000 채널 매일 확인 KST 23:30 진행 -> request 95000 기본
- channel_df: YT 채널 데이터
- loaded_df: 적제 쇼츠 데이터
- basic_shorts_info: 기본적인 Shorts 데이터 추출

In [1]:
# 기존 channel list와 적재 데이터 loaded
import requests
import pandas as pd
from datetime import datetime, timedelta
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# 한국, 미국 데이터 9507개
channel_df = pd.read_excel('us_korea_test.xlsx')[30:40]
# 적재 DB: # -7d, -14d, -21d 만 필터링
loaded_df = pd.read_excel("loaded_db.xlsx")
today = datetime.today()
try:
    loaded_df['check_date'] = loaded_df['check_date'].dt.date
    filter_dates = [today.date() - timedelta(days=7), today.date() - timedelta(days=14), today.date() - timedelta(days=21)]
    # Filter the dataframe based on check_date
    loaded_df = loaded_df[loaded_df['check_date'].isin(filter_dates)]
    # channel_df 와 loaded_df 교집합 데이터
    merged_df = pd.merge(loaded_df, channel_df, on='channel_name', how='inner')
except Exception:
    pass

print(loaded_df)

Empty DataFrame
Columns: [channel_name, shorts_url, shorts_title, shorts_description, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time, shorts_published_date, check_date]
Index: []


In [7]:
# Test 용
# r_full: render 조건 True, request: 10, 쇼츠 데이터
# r_normal: render 조건 False, request: 1, 쇼츠데이터
# r_page: 채널 쇼츠 페이지, reqeust: 1

# scraperAPI result filter with NaN values for missing data
def basic_shorts_info(text):
    # shorts title
    match_title = re.search(r'<title>(.*?) - YouTube</title>', text)
    extracted_title = match_title.group(1) if match_title else np.nan

    # shorts description
    description_pattern = r'attributedDescription":{"content":"(.*?)"'
    # Find the match
    description_match = re.search(description_pattern, text)
    # Extract the content if found
    extracted_description = description_match.group(1) if description_match else None
    
    # shorts total view (date and time)
    date_time_pattern = r'datePublished" content="(\d{4}-\d{2}-\d{2})T(\d{2}):'
    match_date = re.search(date_time_pattern, text)
    extracted_published = f"{match_date.group(1)} {match_date.group(2)}" if match_date else np.nan

    # Thumbnail
    thumbnail_pattern = r'"url":"(https?://[^\s"]+)"'
    thumbnail_match = re.search(thumbnail_pattern, text)
    yt_shorts_thumbnail = thumbnail_match.group(1) if thumbnail_match else np.nan

    # Total view
    view_pattern = r'"viewCountEntityKey":"[^"]+","factoid":\{"factoidRenderer":\{"value":\{"simpleText":"([\d,]+)"'
    view_match = re.search(view_pattern, text)
    extracted_view = view_match.group(1) if view_match else np.nan

    # Total likes
    like_pattern = r'"iconName":"LIKE","title":"([\d.]+)([KM]?)"'
    like_match = re.search(like_pattern, text)
    if like_match:
        extracted_likes = float(like_match.group(1))
        suffix = like_match.group(2)
        if suffix == 'K':
            extracted_likes *= 1000
        elif suffix == 'M':
            extracted_likes *= 1000000
    else:
        extracted_likes = np.nan

    # Total comments    
    comment_num_pattern = r'"text":"Comments"}\]},"contextualInfo":\{"runs":\[\{"text":"([\d.]+)([KM]?)"'
    comment_num_match = re.search(comment_num_pattern, text)
    if comment_num_match:
        extracted_comments_number = float(comment_num_match.group(1))
        suffix = comment_num_match.group(2)
        if suffix == 'K':
            extracted_comments_number *= 1000
        elif suffix == 'M':
            extracted_comments_number *= 1000000
    else:
        extracted_comments_number = np.nan

    # yt shorts comment 결과 및 comment 작성 시간
    comments_time_text_pattern = r'<span[^>]*id="published-time-text"[^>]*>\s*<a[^>]*>\s*(.*?)\s*<\/a>.*?<yt-attributed-string[^>]*>\s*<span[^>]*class="yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap"[^>]*>\s*(.*?)\s*<\/span>'
    
    # Find all matches for time and comments
    comments_time_matches = re.findall(comments_time_text_pattern, text, re.DOTALL)
    
    # Process and clean comments
    extracted_comments_time_matches = []
    if comments_time_matches:
        for time, comment in comments_time_matches:
            # Clean the comment to remove unwanted tags using BeautifulSoup
            soup = BeautifulSoup(comment, 'html.parser')
            cleaned_comment = soup.get_text()
            extracted_comments_time_matches.append((time.strip(), cleaned_comment.strip()))
    else:
        extracted_comments_time_matches = np.nan

    return (extracted_title, extracted_description, extracted_published, yt_shorts_thumbnail, 
            extracted_view, extracted_likes, extracted_comments_number, extracted_comments_time_matches)
# yt_shorts_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r_normal.text)
# print("yt_shorts_title: ", yt_shorts_title)
# print("yt_shorts_description: ", yt_shorts_description)
# print("yt_shorts_published: ", yt_shorts_published)
# print("yt_shorts_thumbnalil: ", yt_shorts_thumbnail)
# print("yt_view: ", yt_shorts_view)
# print("yt_likes: ", yt_shorts_likes)
# print("yt_comments: ", yt_shorts_comments_num)
# print("yt_shorts_comments_time: ", yt_shorts_comments_time)
# print(len(yt_shorts_comments_time))

##### 1. 기존 수집했던 쇼츠 재수집 -7d, -14d, -21d
- loaded_db에서 날짜 필터링해서 데이터 수집 진행

In [8]:
import requests
from datetime import datetime, timedelta

def collecting_past_shorts(api_key, loaded_df):
    today = datetime.today()
    print(loaded_df)
    # Iterate over each row in loaded_df, checking check_date and shorts_url
    for idx, (check_dt, shorts_url) in enumerate(zip(loaded_df['check_date'], loaded_df['shorts_url'])):
        if check_dt == (today.date() - timedelta(days=14)) or check_dt == (today.date() - timedelta(days=21)):
            payload = {
                'api_key': api_key,
                'url': shorts_url
            }
            
            # Send the request
            r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)

            # Extract relevant information from the response
            yt_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r.text)
            # Prepare new row with updated values (keeping the same channel name and other values from loaded_df)
            new_row = {
                'channel_nm': loaded_df.loc[idx, 'channel_nm'], 
                'shorts_url': shorts_url, 
                'shorts_title': yt_title,
                'shorts_description': yt_shorts_description,
                'shorts_thumbnalil': yt_shorts_thumbnail,
                'shorts_view': yt_shorts_view,
                'shorts_likes': yt_shorts_likes,
                'shorts_comments': yt_shorts_comments_num,
                'shorts_comments_time': yt_shorts_comments_time,
                'shorts_published_date': yt_shorts_published,
                'check_date': today.date()  # Use today's date for the new check_date
            }
            
            # Append the new row to the DataFrame
            loaded_df = loaded_df.append(new_row, ignore_index=True)

        # -7d에는 comments 추가 수집
        elif check_dt==(today.date() - timedelta(days=7)): 
            payload = {
                'api_key': API_KEY,
                'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U',
                'render': 'true'
            }

            headers = {
                'x-sapi-render': 'true',
                'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
                # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
                # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
            }
            r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
            yt_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r.text)

            # Prepare new row with updated values (keeping the same channel name and other values from loaded_df)
            new_row = {
                'channel_nm': loaded_df.loc[idx, 'channel_nm'], 
                'shorts_url': shorts_url, 
                'shorts_title': yt_title,
                'shorts_description': yt_shorts_description,
                'shorts_thumbnalil': yt_shorts_thumbnail,
                'shorts_view': yt_shorts_view,
                'shorts_likes': yt_shorts_likes,
                'shorts_comments': yt_shorts_comments_num,
                'shorts_comments_time': yt_shorts_comments_time,
                'shorts_published_date': yt_shorts_published,
                'check_date': today.date()  # Use today's date for the new check_date
            }

            loaded_df = loaded_df.append(new_row, ignore_index=True)

        else:
            print("no data")
            pass

    return loaded_df

collecting_past_shorts(API_KEY, loaded_df)

Empty DataFrame
Columns: [channel_name, shorts_url, shorts_title, shorts_description, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time, shorts_published_date, check_date]
Index: []


,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date


##### 2. channel shorts url input으로 들어갈시 채널 url 필터링(daily)
- 최신 기준으로 맵핑이 되지 않는 경우 최근 1개 url수집 
- 기존 말고 신규 shorts url 확인시 list 추가
- 마지막에 다 확인후 추가된 list 쇼츠 데이터 수집 진행 -> 10 request로

In [9]:
# 매일 전체 채널 필터링
# - 신규 쇼츠 데이터 발견시 list에 추가해서 comments까지 확인
# - 과거 데이터가 없을 수 있으니 만약 하나도 매칭이 안되면 3개 수집, 있다면 그거 앞에꺼를 추가수집
import pandas as pd

def scanning_yt_shorts_page(api_key, channel_db, loaded_db):
    # List to store new shorts results in the format (channel_name, new_shorts_url)
    new_shorts_urls = []
    # List to store channels with no shorts_urls
    no_shorts_list = []

    # Iterate over each row in channel_db
    for index, row in channel_db.iterrows():
        channel_name = row['channel_name']
        yt_fixed_url = row['yt_fixed_url']
        
        # Filter loaded_db where channel_name matches
        matching_rows = loaded_db[loaded_db['channel_name'] == channel_name]
        
        if not matching_rows.empty:
            # Sort by check_date to get the most recent row
            latest_row = matching_rows.sort_values(by='check_date', ascending=False).iloc[0]
        
        # Prepare the payload for ScraperAPI request, using yt_fixed_url
        payload = {
            'api_key': api_key,
            'url': yt_fixed_url
        }
        
        # Send the request to ScraperAPI
        r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
        
        # Define the regex pattern to extract Shorts URLs
        shorts_url_pattern = r'webCommandMetadata":{"url":"(/shorts/\w+)"'
        # Find all matching Shorts URLs
        shorts_urls = re.findall(shorts_url_pattern, r.text)
        shorts_urls =  ['https://www.youtube.com' + url.replace('/shorts/', '/watch?v=') for url in shorts_urls]

        print(f"shorts_urls for {channel_name}: ", shorts_urls)

        # Check if there are any new shorts URLs before the latest one
        if not matching_rows.empty:
            latest_url = latest_row['shorts_url']
            # If the latest URL exists in the list of Shorts URLs, find the ones before it
            if latest_url in shorts_urls:
                matching_index = shorts_urls.index(latest_url)
                # Collect all URLs before the match with channel_name
                for shorts_url in shorts_urls[:matching_index]:
                    new_shorts_urls.append((channel_name, shorts_url))
        else:
            # If no match in loaded_db, return the first shorts URL with channel_name
            if shorts_urls:
                new_shorts_urls.append((channel_name, shorts_urls[0]))
        
        # If shorts_urls is empty, add the channel_name and yt_fixed_url to the no_shorts_list
        if not shorts_urls:
            no_shorts_list.append((channel_name, yt_fixed_url))
    
    # Convert the no_shorts_list to a DataFrame
    no_shorts_df = pd.DataFrame(no_shorts_list, columns=['channel_name', 'yt_fixed_url'])

    return new_shorts_urls, no_shorts_df  # Return both the list and DataFrame


In [10]:
new_shorts_urls = scanning_yt_shorts_page('3397e8a6a37e89b03b08750a27575df8', channel_df, loaded_df)
print(new_shorts_urls)

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Marc Brunet:  ['https://www.youtube.com/watch?v=Qc6NO19u3MM', 'https://www.youtube.com/watch?v=qSfXMQQRNY0', 'https://www.youtube.com/watch?v=QcnpGUdGFmg', 'https://www.youtube.com/watch?v=wMXHYET8xP8', 'https://www.youtube.com/watch?v=SRzUqX9pza8', 'https://www.youtube.com/watch?v=UituSM5LlZc', 'https://www.youtube.com/watch?v=8sORtqjvClM', 'https://www.youtube.com/watch?v=t51uvIVlkwg', 'https://www.youtube.com/watch?v=kOyXKhVChTo', 'https://www.youtube.com/watch?v=TTU6VNh07aM', 'https://www.youtube.com/watch?v=11rLjEFozYY', 'https://www.youtube.com/watch?v=ryGJf_v4WHE', 'https://www.youtube.com/watch?v=PNIL5KP51N4', 'https://www.youtube.com/watch?v=83CEXyazHcc', 'https://www.youtube.com/watch?v=G4PoTrGsEOg', 'https://www.youtube.com/watch?v=IaOPd8Q9_MI', 'https://www.youtube.com/watch?v=_cpmZJ2CqHQ', 'https://www.youtube.com/watch?v=1dQW5oshfTM', 'https://www.youtube.com/watch?v=aI3mrZo4Hxc', 'https://www.youtube.com/watch?v=6j9GT29xHOI', 'https://www.youtube.com/watc

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Jen_ny69:  ['https://www.youtube.com/watch?v=ixq_pOK7jk0', 'https://www.youtube.com/watch?v=2iRkPzWEwoM', 'https://www.youtube.com/watch?v=12cAOIfgpN8', 'https://www.youtube.com/watch?v=olKeodW3OgI', 'https://www.youtube.com/watch?v=bdWLh27gMCg', 'https://www.youtube.com/watch?v=u2JvUYlHcvw', 'https://www.youtube.com/watch?v=ubl0lfLL1BA', 'https://www.youtube.com/watch?v=DNd49qvBXa8', 'https://www.youtube.com/watch?v=pxKm4bgE1A8', 'https://www.youtube.com/watch?v=0Vhy78hFn0M', 'https://www.youtube.com/watch?v=ul0n6QL9CDg', 'https://www.youtube.com/watch?v=iEFgA507ubY', 'https://www.youtube.com/watch?v=dFlGiETcf_w', 'https://www.youtube.com/watch?v=LmP9AwSqgPk', 'https://www.youtube.com/watch?v=omejymfxZ8w', 'https://www.youtube.com/watch?v=QEAALeEsMVI', 'https://www.youtube.com/watch?v=lSUKtugjzUY', 'https://www.youtube.com/watch?v=zW8EYv6UUeU', 'https://www.youtube.com/watch?v=3xX7yJf9Ii4', 'https://www.youtube.com/watch?v=N2YWU5bPf98', 'https://www.youtube.com/watch?v

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

shorts_urls for Brian Tracy:  ['https://www.youtube.com/watch?v=C6HaBSkA6xU', 'https://www.youtube.com/watch?v=WM1KN4J05FM', 'https://www.youtube.com/watch?v=pkDnvxaPJtU', 'https://www.youtube.com/watch?v=L61s3UPCDQA', 'https://www.youtube.com/watch?v=B_VTA8X9y3U', 'https://www.youtube.com/watch?v=Ve7q421iE2w', 'https://www.youtube.com/watch?v=7oFeHZQ3n4M', 'https://www.youtube.com/watch?v=S1SfRCRbaf0', 'https://www.youtube.com/watch?v=OH2V71GWgUA', 'https://www.youtube.com/watch?v=aaMcWg_XEVU', 'https://www.youtube.com/watch?v=kjRDvACn6Ls', 'https://www.youtube.com/watch?v=ZBvv6ciCVFk', 'https://www.youtube.com/watch?v=ASqkHoHfDMI', 'https://www.youtube.com/watch?v=7oKpedyMLJw', 'https://www.youtube.com/watch?v=9JzJDMt7eXE', 'https://www.youtube.com/watch?v=niI8RlDiLW0', 'https://www.youtube.com/watch?v=crZG3xDWA_A', 'https://www.youtube.com/watch?v=bz9wtjo0DVQ', 'https://www.youtube.com/watch?v=pSZLAXPhPXc', 'https://www.youtube.com/watch?v=YBhM2nWb0wA', 'https://www.youtube.com/watc

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Now You See It:  ['https://www.youtube.com/watch?v=g6lcT4EhNzM']


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Fitness Guru:  []
([('Marc Brunet', 'https://www.youtube.com/watch?v=Qc6NO19u3MM'), ('Jen_ny69', 'https://www.youtube.com/watch?v=ixq_pOK7jk0'), ('DawateIslami', 'https://www.youtube.com/watch?v=JhocFEigq3k'), ('Go Natural English with Gabby Wallace', 'https://www.youtube.com/watch?v=v9dcf7OF1Wg'), ('ReligionForBreakfast', 'https://www.youtube.com/watch?v=WZn6dF9C1NQ'), ('Medicosis Perfectionalis', 'https://www.youtube.com/watch?v=wFVlzZUWFcw'), ('理科太太 Li Ke Tai Tai', 'https://www.youtube.com/watch?v=E1789fuBkkU'), ('Brian Tracy', 'https://www.youtube.com/watch?v=C6HaBSkA6xU'), ('Now You See It', 'https://www.youtube.com/watch?v=g6lcT4EhNzM')],    channel_name                                       yt_fixed_url
0  Fitness Guru  https://www.youtube.com/channel/UCKJOEv8ymPEmT...)


#### YouTube 채널 및 shorts를 업로드 하지 않는 경우
- data_yn에서 y를 n으로 변경 (y가 default)
- 필터링 중 n이였던 건이 y가 되는 경우도 체크

In [20]:
def update_channel_df_with_new_shorts(channel_df, new_shorts_urls_df):
    # Iterate over each row in new_shorts_urls_df to match with channel_df
    for index, row in new_shorts_urls_df.iterrows():
        target_channel_name = row['channel_name']
        
        # Update 'data_yn' in channel_df where 'channel_name' matches
        channel_df.loc[channel_df['channel_name'] == target_channel_name, 'data_yn'] = 'n'

    # Return the updated channel_df with all rows
    return channel_df

updated_channel_df = update_channel_df_with_new_shorts(channel_df, new_shorts_urls[1])

# Display the updated DataFrame
print(updated_channel_df)

                                       youtuberme_url   category  \
30  https://us.youtubers.me//brushboost/youtuber-s...  education   
31   https://us.youtubers.me//jen_ny69/youtuber-stats  education   
32  https://us.youtubers.me//dawateislami/youtuber...  education   
33  https://us.youtubers.me//go-natural-english-by...  education   
34  https://us.youtubers.me//religionforbreakfast/...  education   
35  https://us.youtubers.me//medicosis-perfectiona...  education   
36  https://us.youtubers.me//tai-tai-li-ke/youtube...  education   
37  https://us.youtubers.me//brian-tracy/youtuber-...  education   
38  https://us.youtubers.me//now-you-see-it/youtub...  education   
39  https://us.youtubers.me//fitness-guru/youtuber...  education   

          country                           channel_name  subscribers  \
30  United States                            Marc Brunet      1570000   
31  United States                               Jen_ny69      1020000   
32  United States               

In [15]:
new_shorts_urls[1]

,channel_name,yt_fixed_url
0,Fitness Guru,https://www.youtube.com/channel/UCKJOEv8ymPEmT...


##### 3. 수집한 channel list 수집 후 loaded_db에 적재
- new_shorts_urls 기준으로 rendered=True로 수집
- 수집 데이터 loaded_db에 적재 후 저장

In [22]:
# url 여러개 진행
import requests
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
today = datetime.today()
def add_new_shorts_data(api_key, urls):
    data_list = []
    check_date = today
    # Separate channel names and shorts URLs
    channel_names = [item[0] for item in urls]
    shorts_urls = [item[1] for item in urls]
    
    for channel_name, url in zip(channel_names, shorts_urls):
        print(channel_name, url)
        shorts_url = url
        # Prepare payload and headers for the request
        payload = {
            'api_key': api_key,
            'url': url
            # ,'render': 'true'
        }

        headers = {
            'x-sapi-render': 'true',
            'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 3}]'
        }

        # Make the GET request
        r = requests.get('https://api.scraperapi.com/', params=payload,  verify=False)#headers=headers,
        # Extract the necessary data
        shorts_title, shorts_description, shorts_published, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time = basic_shorts_info(r.text)
        
        # Print the extracted information for debugging
        print("title: ", channel_name)
        print("shorts_title: ", shorts_title)
        print("shorts_description: ", shorts_description)
        print("shorts_published: ", shorts_published)
        print("shorts_thumbnalil: ", shorts_thumbnail)
        print("view: ", shorts_view)
        print("likes: ", shorts_likes)
        print("comments: ", shorts_comments_num)
        print("shorts_comments_time: ", shorts_comments_time)

        # Append as a tuple to the data_list
        data_list.append((
            channel_name,
            shorts_url, 
            shorts_title,  
            shorts_description, 
            shorts_thumbnail, 
            shorts_view, 
            shorts_likes, 
            shorts_comments_num, 
            shorts_comments_time,
            shorts_published, 
            check_date

        ))
    
    return data_list

data_list = add_new_shorts_data(API_KEY, new_shorts_urls[0])
print(data_list)



Marc Brunet https://www.youtube.com/watch?v=Qc6NO19u3MM


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Marc Brunet
shorts_title:  
shorts_description:  👨‍🎨 Serious about your art? Check out my ART School program here! ➡️ cgart.school (or link in bio)
shorts_published:  nan
shorts_thumbnalil:  https://support.google.com/youtube/answer/3037019#zippy=%2Ccheck-that-youre-signed-into-youtube
view:  121,372
likes:  17000.0
comments:  94.0
shorts_comments_time:  nan
Jen_ny69 https://www.youtube.com/watch?v=ixq_pOK7jk0


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Jen_ny69
shorts_title:  
shorts_description:  Watch full video on ‪@adayinthelifeofa69‬
shorts_published:  nan
shorts_thumbnalil:  https://support.google.com/youtube/answer/3037019#zippy=%2Ccheck-that-youre-signed-into-youtube
view:  6,270
likes:  328.0
comments:  13.0
shorts_comments_time:  nan
DawateIslami https://www.youtube.com/watch?v=JhocFEigq3k


...

title:  理科太太 Li Ke Tai Tai
shorts_title:  貓咪個性這麼差，原來都是基因決定的！
shorts_description:  None
shorts_published:  2024-01-17 22
shorts_thumbnalil:  https://rr3---sn-vgqsrnsr.googlevideo.com/videoplayback?expire=1729795493\u0026ei=RUEaZ4bwA9ijlu8P0YPk0Q8\u0026ip=109.121.40.7\u0026id=o-AN4g-UD0h2UxQRyKNINNY7tTownovj-xCztRKcylTZOv\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1729773893%2C\u0026mh=NB\u0026mm=31%2C29\u0026mn=sn-vgqsrnsr%2Csn-vgqsknze\u0026ms=au%2Crdu\u0026mv=u\u0026mvi=3\u0026pl=24\u0026rms=au%2Cau\u0026bui=AXLXGFTQuTCmAdRYUykqKCt8M5xY0FBSB18gB3-ulcEq6Yy1fOdbocNhKWR833ZxQLuAzvH9TqbqUf0g\u0026spc=54MbxebnlHrjXAAW578iPCXkhvWIjQdcbyLTAisLS52OcZ7wfTCXzGdbUs350hE\u0026vprv=1\u0026svpuc=1\u0026mime=video%2Fmp4\u0026ns=PHlJtvMFgu_H_WM72L_Kw8QQ\u0026rqh=1\u0026gir=yes\u0026clen=2152144\u0026ratebypass=yes\u0026dur=28.885\u0026lmt=1705556810718508\u0026mt=1729773546\u0026fvip=1\u0026fexp=51312688%2C51326932\u0026c=WEB\u0026sefc=1\u0026txp=6300224\u

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Brian Tracy
shorts_title:  Shift Your Mindset
shorts_description:  Imagine the possibilities if failure weren’t a factor—how bold would your dreams be? Often, we hold back due to fear, but envisioning a life free from this constraint can reveal passions we’ve sidelined. 💭⁠\n⁠\nThis mindset inspires us to think big, whether it’s starting a business, pursuing a creative project, or making a significant life change. It helps us identify what truly matters and motivates us to take actionable steps toward our dreams.⁠\n⁠\nSo, what dreams are calling you, and what steps can you take today to make them a reality?⁠\n\nCheck out this 14-Step Goal Setting Guide to help set, pursue, and achieve your goals: https://bit.ly/4bbccaa\n⁠\n🎥: @hustlewithlana ⁠on Instagram\n\n#briantracy #success #motivation #goals #inspiration #stayinspired #successtips #motivated #successful \n\n___________________\n\nLearn more: \n\nSubscribe to my channel for free offers, tips and more!\nYouTube: http://ow.ly

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


title:  Now You See It
shorts_title:  The Best Steven Spielberg Interview Question Ever #shorts #film #movies #stevenspielberg
shorts_description:  Movies: The Fablemans (2022) and Close Encounters of the Third Kind (1977)\nFrom James Lipton - Inside Actor's Studio Episode #5.9\nFull interview: https://www.dailymotion.com/video/xjqe5\nOriginal interview source:    • Best Interview Question Ever - Steven...  \n\n#shorts #film #movies #stevenspielberg
shorts_published:  2023-01-07 09
shorts_thumbnalil:  https://rr1---sn-f58xn2xxq-aj5l.googlevideo.com/videoplayback?expire=1729795501\u0026ei=TUEaZ_CCL8zYxN8P7uub-QM\u0026ip=103.80.10.244\u0026id=o-AMxyITgrENRuzdK3vFcXUvlcAWFyxr8xCIiHcWy2y-o8\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1729773901%2C\u0026mh=R1\u0026mm=31%2C29\u0026mn=sn-f58xn2xxq-aj5l%2Csn-hgn7rnll\u0026ms=au%2Crdu\u0026mv=m\u0026mvi=1\u0026pl=23\u0026rms=au%2Cau\u0026initcwndbps=726250\u0026bui=AXLXGFS7xAEJOiLKutq64K3bZQWf9UoJTNjc

In [25]:
# 추가된 데이터 dataframe으로 loaed_db에 추가
def add_new_data_to_loaded_db(data_list, loaded_db):
    # Define the columns for the DataFrame
    columns = [
        'channel_name', 'shorts_url', 'shorts_title', 'shorts_description', 
        'shorts_thumbnail', 'shorts_view', 'shorts_likes', 
        'shorts_comments_num', 'shorts_comments_time', 
        'shorts_published_date', 'check_date'
    ]
    
    # Map the data_list to the correct columns and create a new DataFrame
    new_data_df = pd.DataFrame(data_list, columns=columns)
    
    # Append the new data to the loaded_db DataFrame
    updated_loaded_db = pd.concat([loaded_db, new_data_df], ignore_index=True)
    
    return updated_loaded_db

loaded_df = add_new_data_to_loaded_db(data_list, loaded_df)
loaded_df

C:\Users\T14Gen1\AppData\Local\Temp\ipykernel_32672\1389024915.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_loaded_db = pd.concat([loaded_db, new_data_df], ignore_index=True)


,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date
0,Marc Brunet,https://www.youtube.com/watch?v=Qc6NO19u3MM,,👨‍🎨 Serious about your art? Check out my ART S...,https://support.google.com/youtube/answer/3037...,"121,372",17000.0,94.0,NaN,NaN,2024-10-24 21:44:30.453033
1,Jen_ny69,https://www.youtube.com/watch?v=ixq_pOK7jk0,,Watch full video on ‪@adayinthelifeofa69‬,https://support.google.com/youtube/answer/3037...,"6,270",328.0,13.0,NaN,NaN,2024-10-24 21:44:30.453033
2,DawateIslami,https://www.youtube.com/watch?v=JhocFEigq3k,,Kabhi Ye Bhi Try Karen | #dawateislami #whatsa...,https://support.google.com/youtube/answer/3037...,926,229.0,3.0,NaN,NaN,2024-10-24 21:44:30.453033
3,Go Natural English with Gabby Wallace,https://www.youtube.com/watch?v=v9dcf7OF1Wg,,None,https://support.google.com/youtube/answer/3037...,"3,897",72.0,3.0,NaN,NaN,2024-10-24 21:44:30.453033
4,ReligionForBreakfast,https://www.youtube.com/watch?v=WZn6dF9C1NQ,,None,https://support.google.com/youtube/answer/3037...,"82,627",4700.0,206.0,NaN,NaN,2024-10-24 21:44:30.453033
5,Medicosis Perfectionalis,https://www.youtube.com/watch?v=wFVlzZUWFcw,,None,https://support.google.com/youtube/answer/3037...,"2,977",284.0,5.0,NaN,NaN,2024-10-24 21:44:30.453033
6,理科太太 Li Ke Tai Tai,https://www.youtube.com/watch?v=E1789fuBkkU,貓咪個性這麼差，原來都是基因決定的！,None,https://rr3---sn-vgqsrnsr.googlevideo.com/vide...,"15,447",327.0,12.0,NaN,2024-01-17 22,2024-10-24 21:44:30.453033
7,Brian Tracy,https://www.youtube.com/watch?v=C6HaBSkA6xU,Shift Your Mindset,Imagine the possibilities if failure weren’t a...,https://i.ytimg.com/vi/C6HaBSkA6xU/hq2.jpg?sqp...,"4,477",562.0,8.0,NaN,2024-10-23 09,2024-10-24 21:44:30.453033
8,Now You See It,https://www.youtube.com/watch?v=g6lcT4EhNzM,The Best Steven Spielberg Interview Question E...,Movies: The Fablemans (2022) and Close Encount...,https://rr1---sn-f58xn2xxq-aj5l.googlevideo.co...,"47,959",2400.0,32.0,NaN,2023-01-07 09,2024-10-24 21:44:30.453033


In [133]:
len(loaded_df.columns)
len(data_list)

2

#### loaded_df 저장

In [26]:
# 중복 제거
loaded_df = loaded_df.drop_duplicates()
loaded_df.to_excel("loaded_db.xlsx", index=False)
